#### Model Evaluation and Refinement

In [ ]:
# Standard imports
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
sns.set()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DA0101EN-SkillsNetwork/labs/Data%20files/module_5_auto.csv', header=0)

In [ ]:
df.head()

First, let's only use numeric data:

In [ ]:
df_numeric = df._get_numeric_data()

In [ ]:
df_numeric.head()

In [ ]:
df_numeric.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df_numeric.head()

Libraries for plotting:

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual

Functions for Plotting

In [ ]:
def DistributionPlot(RedFunction, BlueFunction, RedName, BlueName, Title):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))

    ax1 = sns.distplot(RedFunction, hist=False, color="r", label=RedName)
    ax2 = sns.distplot(BlueFunction, hist=False,
                       color="b", label=BlueName, ax=ax1)

    plt.title(Title)
    plt.xlabel('Price (in dollars)')
    plt.ylabel('Proportion of Cars')

    plt.show()
    plt.close()


def PollyPlot(xtrain, xtest, y_train, y_test, lr, poly_transform):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))

    # training data
    # testing data
    # lr:  linear regression object
    # poly_transform:  polynomial transformation object

    xmax = max([xtrain.values.max(), xtest.values.max()])

    xmin = min([xtrain.values.min(), xtest.values.min()])

    x = np.arange(xmin, xmax, 0.1)

    plt.plot(xtrain, y_train, 'ro', label='Training Data')
    plt.plot(xtest, y_test, 'go', label='Test Data')
    plt.plot(x, lr.predict(poly_transform.fit_transform(
        x.reshape(-1, 1))), label='Predicted Function')
    plt.ylim([-10000, 60000])
    plt.ylabel('Price')
    plt.legend()

##### Part 1: Training and Testing

In [ ]:
y_data = df_numeric['price']

In [ ]:
x_data = df_numeric.drop('price', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.10, random_state=1)

print("number of test samples :", x_test.shape[0])
print("number of training samples:", x_train.shape[0])

In [ ]:
x_train1, x_test1, y_train1, y_test1 = train_test_split(
    x_data, y_data, test_size=0.40, random_state=0)

print("number of test samples :", x_test1.shape[0])
print("number of training samples:", x_train1.shape[0])

In [ ]:
from sklearn.linear_model import LinearRegression
lre = LinearRegression()

In [ ]:
lre.fit(x_train[['horsepower']], y_train)

In [ ]:
lre.score(x_test[['horsepower']], y_test)

In [ ]:
lre.score(x_train[['horsepower']], y_train)

In [ ]:
lre.score(x_test1[['horsepower']], y_test1)

In [ ]:
lre.score(x_train1[['horsepower']], y_train1)

##### Cross-Validation Score
Sometimes you don't have enough data for testing.\
As a result, you may want to perform cross-validaion

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
Rcross = cross_val_score(lre, x_data[['horsepower']], y_data, cv=4)
Rcross

In [ ]:
Rcross.mean()

In [ ]:
Rcross.std()

We can use negative squared error as a score by setting the parameter 'scoring' metric to 'neg_mean_squared_error'

In [ ]:
-1 * cross_val_score(lre,  x_data[['horsepower']],
                     y_data, cv=4, scoring='neg_mean_squared_error')

In [ ]:
Rc = cross_val_score(lre, x_data[['horsepower']], y_data, cv=2)
Rc.mean()

You can also use the function 'cross_val_predict' to predict the output. The \
function splits up the data into the specified number of folds, with one \
fold for testing and the other folds are used for training. First, import the function:

In [ ]:
from sklearn.model_selection import cross_val_predict

In [ ]:
yhat = cross_val_predict(lre, x_data[['horsepower']], y_data, cv=4)

In [ ]:
yhat[0:5]

##### Part 2: Overfitting, Underfitting and Model Selection

Let's create Multiple Linear Regression objects and train the model using 'horsepower', 'curb-weight', 'engine-size' and 'highway-mpg' as features.

In [ ]:
lr = LinearRegression()
lr.fit(x_train[['horsepower', 'curb-weight',
       'engine-size', 'highway-mpg']], y_train)

In [ ]:
yhat_train = lr.predict(x_train[['horsepower', 'curb-weight',
                                 'engine-size', 'highway-mpg']])
yhat_train[0:5]

In [ ]:
yhat_test = lr.predict(x_test[['horsepower', 'curb-weight',
                               'engine-size', 'highway-mpg']])
yhat_test[0:5]

We can perform some model evaluation using the training and testing data separately\
But 1st let's do some plotting

In [ ]:
Title = 'Distribution  Plot of  Predicted Value Using Training Data vs Training Data Distribution'
DistributionPlot(y_train, yhat_train, "Actual Values (Train)",
                 "Predicted Values (Train)", Title)

We see the model is good at predicting seen data(train data). Let us now see test data

In [ ]:
Title = 'Distribution  Plot of  Predicted Value Using Testing Data vs Testing-Data Data Distribution'
DistributionPlot(y_test, yhat_test, "Actual Values (Test)",
                 "Predicted Values (Test)", Title)


Comparing Figure 1 and Figure 2, it is evident that the distribution of the test data in Figure 1 is much better at fitting the data. \
This difference in Figure 2 is apparent in the range of 5000 to 15,000. This is where the shape of the distribution is extremely different. \
Let's see if polynomial regression also exhibits a drop in the prediction accuracy when analysing the test dataset

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

Let's create a polynomial of degree 5

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.45, random_state=0)

In [ ]:
pr = PolynomialFeatures(5)
x_train_pr = pr.fit_transform(x_train[['horsepower']])
x_test_pr = pr.fit_transform(x_test[['horsepower']])
pr

In [ ]:
poly = LinearRegression()
poly.fit(x_train_pr, y_train)

In [ ]:
yhat = poly.predict(x_test_pr)
yhat[0:5]

In [ ]:
print("Predicted values:", yhat[0:4])
print("True values:", y_test[0:4].values)

We will use the function "PollyPlot" that we defined at the beginning to display the training data, testing data, and the predicted function.

In [ ]:
PollyPlot(x_train[['horsepower']], x_test[['horsepower']],
          y_train, y_test, poly, pr)

We see that the estimated function appears to track the data but around 200 horsepower, the function begins to diverge from the data points.


In [ ]:
poly.score(x_train_pr, y_train)

In [ ]:
poly.score(x_test_pr, y_test)

A -ve R-score is a sign of overfitting

Let us see how the R^2 changes on the test data for different polynomials

In [ ]:
Rsqu_test = []

order = [1, 2, 3, 4]

for n in order:
    pr = PolynomialFeatures(n)

    x_train_pr = pr.fit_transform(x_train[['horsepower']])
    x_test_pr = pr.fit_transform(x_test[['horsepower']])

    lr.fit(x_train_pr, y_train)

    Rsqu_test.append(lr.score(x_test_pr, y_test))

# Visualizing our results
plt.plot(order, Rsqu_test)
plt.xlabel('order')
plt.ylabel('R^2')
plt.title('R^2 Using Test Data')
plt.text(3, 0.75, 'Maximum R^2 ')

In [ ]:
def f(degree, test_data):
    x_train, x_test, y_train, y_test = train_test_split(
        x_data, y_data, test_size=test_data, random_state=0)
    pr = PolynomialFeatures(degree=degree)
    x_train_pr = pr.fit_transform(x_train[['horsepower']])
    x_test_pr = pr.fit_transform(x_test[['horsepower']])
    poly = LinearRegression()
    poly.fit(x_train_pr, y_train)
    PollyPlot(x_train['horsepower'], x_test['horsepower'],
              y_train, y_test, poly, pr)

We can create an interface that allows one to experiment with different polynomial orders and different amount of data

In [ ]:
interact(f, degree=(0, 6, 1), test_data=(0.05, 0.95, 0.05))

We can perform polynomial transformations with more than one feature. e.g

In [ ]:
pr1 = PolynomialFeatures(2)

In [ ]:
x_train_pr1 = pr1.fit_transform(
    x_train[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']])
x_test_pr1 = pr1.fit_transform(
    x_test[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']])

In [ ]:
x_train_pr1.shape

Wr can see that transforming using the Polynomial basis function results to a matrix with 15 features from 4

In [ ]:
poly1 = lr.fit(x_train_pr1, y_train)

In [ ]:
Title = 'Distribution  Plot of  Predicted Value Using Testing Data vs Actual Testing-Data Data Distribution'
# DistributionPlot(y_test, yhat_test, "Actual Values (Train)",
#                  "Predicted Values (Train)", Title)
yhat_test1 = poly1.predict(x_test_pr1)
DistributionPlot(y_test, yhat_test1, "Actual Values (Test)",
                 "Predicted Values (Test)", Title)

The predicted value is higher than actual value for cars where the price $10,000 range, conversely the predicted price is lower than the price cost in the $30,000 to $40,000 range. As such the model is not as accurate in these ranges.

##### Part 3 : Ridge Regression

In [ ]:
pr = PolynomialFeatures(degree=2)
x_train_pr = pr.fit_transform(
    x_train[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg', 'normalized-losses', 'symboling']])
x_test_pr = pr.fit_transform(
    x_test[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg', 'normalized-losses', 'symboling']])

In [ ]:
from sklearn.linear_model import Ridge

Let's create a Ridge regression object and set alpha to 0.1

In [ ]:
RidgeModel = Ridge(alpha=0.1)

In [ ]:
RidgeModel.fit(x_train_pr, y_train)

In [ ]:
yhat = RidgeModel.predict(x_test_pr)

In [ ]:
print('predicted:', yhat[0:4])
print('test set :', y_test[0:4].values)

We select the value of alpha that minimizes the test error. To do so, we can use a for loop. We have also created a progress bar to see how many iterations we have completed so far.

In [ ]:
from tqdm import tqdm

Rsqu_test = []
Rsqu_train = []
dummy1 = []
Alpha = 10 * np.array(range(0, 1000))
pbar = tqdm(Alpha)

for alpha in pbar:
    RigeModel = Ridge(alpha=alpha)
    RigeModel.fit(x_train_pr, y_train)
    test_score, train_score = RigeModel.score(
        x_test_pr, y_test), RigeModel.score(x_train_pr, y_train)

    pbar.set_postfix({"Test Score": test_score, "Train Score": train_score})

    Rsqu_test.append(test_score)
    Rsqu_train.append(train_score)

We can visulaize our findings

In [ ]:
width = 12
height = 10

plt.figure(figsize=(width, height))
plt.plot(Alpha, Rsqu_test, label='validation data  ')
plt.plot(Alpha, Rsqu_train, 'r', label='training Data ')
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.legend()

The red line in Figure above represents the R^2 of the training data. As alpha increases the R^2 decreases. Therefore, as alpha increases, the model performs worse on the training data

The blue line represents the R^2 on the validation data. As the value for alpha increases, the R^2 increases and converges at a point.

In [ ]:
rm = Ridge(alpha=10)

In [ ]:
rm.fit(x_train_pr, y_train)
rm.score(x_test_pr, y_test)

##### Part 4 : Grid Search

The term alpha is a hyperparameter. Sklearn has the class GridSearchCV to make the process of finding the best hyperparameter simpler.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Creating a dict of parameters to be passed to the grid
parameters1 = [
    {'alpha': [0.001, 0.1, 1, 10, 100, 1000, 10000, 100000, 100000]}]
parameters1

In [ ]:
RR = Ridge()

In [ ]:
grid1 = GridSearchCV(RR, parameters1, cv=4)

In [ ]:
grid1.fit(x_data[['horsepower', 'curb-weight',
          'engine-size', 'highway-mpg']], y_data)

In [ ]:
BestRR = grid1.best_estimator_
BestRR

This shows that the best alpha is 10000, let's see how the best model performs to our test data

In [ ]:
BestRR.score(x_test[['horsepower', 'curb-weight',
             'engine-size', 'highway-mpg']], y_test)

With the r^2 score up to 84%, one can see that using Grid search, we are able to find the best hyperparams more easily.